In [1]:
import pandas as pd
import numpy as np
import pickle as pkl
import os
import socket
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
from bmtk.simulator.filternet.lgnmodel.fitfuns import makeBasis_StimKernel
from bmtk.simulator.filternet.lgnmodel.spatialfilter import GaussianSpatialFilter
from bmtk.simulator.filternet.lgnmodel.temporalfilter import TemporalFilterCosineBump
from bmtk.simulator.filternet.lgnmodel.util_fns import get_data_metrics_for_each_subclass, \
    get_tcross_from_temporal_kernel

In [12]:
path='spontaneous_firing_rates.pkl'
with open(path, 'rb') as f:
    data = pkl.load(f)
data

array([1.76666667, 1.76666667, 1.76666667, ..., 3.62      , 3.62      ,
       3.62      ])

In [13]:
path='temporal_kernels.pkl'
with open(path, 'rb') as f:
    data = pkl.load(f)
data

{'dom_temporal_kernels': array([[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 9.8227727e-05,
         0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.6836631e-03,
         0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 8.1802916e-04,
         0.0000000e+00, 0.0000000e+00],
        ...,
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
         0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
         0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
         0.0000000e+00, 0.0000000e+00]], dtype=float32),
 'non_dom_temporal_kernels': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 

In [4]:
path='data/lgn_full_col_cells_3.csv'
d = pd.read_csv(path, delimiter=' ')

In [10]:
spatial_sizes = d['spatial_size'].to_numpy()
model_id = d['model_id'].to_numpy()
amplitude = np.array(
            [1. if a.count('ON') > 0 else -1. for a in model_id])
non_dom_amplitude = np.zeros_like(amplitude)
is_composite = np.array([a.count('ON') > 0 and a.count(
            'OFF') > 0 for a in model_id]).astype(np.float32)
x = d['x'].to_numpy()
y = d['y'].to_numpy()
non_dominant_x = np.zeros_like(x)
non_dominant_y = np.zeros_like(y)
tuning_angle = d['tuning_angle'].to_numpy()
subfield_separation = d['sf_sep'].to_numpy()


In [19]:
s_path = 'spontaneous_firing_rates.pkl'
with open(s_path, 'rb') as f:
    spontaneous_firing_rates = pkl.load(f)

In [22]:
spontaneous_firing_rates

array([1.76666667, 1.76666667, 1.76666667, ..., 3.62      , 3.62      ,
       3.62      ])

In [29]:
temporal_peaks_dom = np.stack(
            (d['kpeaks_dom_0'].to_numpy(), d['kpeaks_dom_1'].to_numpy()), -1)
temporal_weights = np.stack(
            (d['weight_dom_0'].to_numpy(), d['weight_dom_1'].to_numpy()), -1)
temporal_delays = np.stack(
            (d['delay_dom_0'].to_numpy(), d['delay_dom_1'].to_numpy()), -1)

In [30]:
temporal_peaks_non_dom = np.stack(
            (d['kpeaks_non_dom_0'].to_numpy(), d['kpeaks_non_dom_1'].to_numpy()), -1)
temporal_weights_non_dom = np.stack(
    (d['weight_non_dom_0'].to_numpy(), d['weight_non_dom_1'].to_numpy()), -1)
temporal_delays_non_dom = np.stack(
    (d['delay_non_dom_0'].to_numpy(), d['delay_non_dom_1'].to_numpy()), -1)


In [32]:
# count the number of non nan rows in temporal_weights_non_dom
num_non_dom = np.sum(~np.isnan(temporal_weights_non_dom[:, 0]))
print('Number of non dominant cells: {}'.format(num_non_dom))

Number of non dominant cells: 1950


In [34]:
# values from bmtk
t_path = 'temporal_kernels.pkl'
with open(t_path, 'rb') as f:
    loaded = pkl.load(f)

In [36]:
dom_temporal_kernels = loaded['dom_temporal_kernels']
non_dom_temporal_kernels = loaded['non_dom_temporal_kernels']
non_dominant_x = loaded['non_dominant_x']
non_dominant_y = loaded['non_dominant_y']
amplitude = loaded['amplitude']
non_dom_amplitude = loaded['non_dom_amplitude']
spontaneous_firing_rates = loaded['spontaneous_firing_rates']

In [41]:
spontaneous_firing_rates.shape

(17400,)

In [46]:
np.sum(np.cumsum(np.abs(dom_temporal_kernels), axis=1) <= 1e-6, 1)

array([565, 564, 563, ..., 480, 485, 478])

In [47]:
truncation = np.min(
    np.sum(np.cumsum(np.abs(dom_temporal_kernels), axis=1) <= 1e-6, 1))
non_dom_truncation = np.min(
    np.sum(np.cumsum(np.abs(non_dom_temporal_kernels), axis=1) <= 1e-6, 1))
truncation = np.min([truncation, non_dom_truncation])
print(f'Could truncate {truncation} steps from filter')

Could truncate 126 steps from filter


In [50]:
x = x * 239 / 240
# print range of x
print('x range: {} - {}'.format(np.min(x), np.max(x)))

# repeat the same for y
y = y * 119 / 120
print('y range: {} - {}'.format(np.min(y), np.max(y)))

x range: 0.00019167014462973705 - 236.97845715910154
y range: 0.0003447157278895983 - 118.9986440607937


In [52]:
non_dominant_x = non_dominant_x * 239 / 240
non_dominant_y = non_dominant_y * 119 / 120
non_dominant_x[np.floor(non_dominant_x) < 0] = 0.
non_dominant_y[np.floor(non_dominant_y) < 0] = 0.

0

In [14]:
d_spatial = 1.
spatial_range = np.arange(0, 15, d_spatial)
x_range = np.arange(-50, 51)
y_range = np.arange(-50, 51)
# kernels = []
gaussian_filters = []
for i in range(len(spatial_range) - 1):
    sigma = np.round(np.mean(spatial_range[i:i+2])) / 3
    original_filter = GaussianSpatialFilter(translate=(
        0., 0.), sigma=(sigma, sigma), origin=(0., 0.))
    kernel = original_filter.get_kernel(
        x_range, y_range, amplitude=1.).full()
    nonzero_inds = np.where(np.abs(kernel) > 1e-9)
    rm, rM = nonzero_inds[0].min(), nonzero_inds[0].max()
    cm, cM = nonzero_inds[1].min(), nonzero_inds[1].max()
    kernel = kernel[rm:rM + 1, cm:cM + 1]
    print(kernel.shape)

(1, 1)
(7, 7)
(7, 7)
(11, 11)
(11, 11)
(17, 17)
(17, 17)
(23, 23)
(23, 23)
(27, 27)
(27, 27)
(33, 33)
(33, 33)
(39, 39)


In [57]:
kernel.shape

(101, 101)

In [59]:
nonzero_inds = np.where(np.abs(kernel) > 1e-9)
rm, rM = nonzero_inds[0].min(), nonzero_inds[0].max()
cm, cM = nonzero_inds[1].min(), nonzero_inds[1].max()
kernel = kernel[rm:rM + 1, cm:cM + 1]

In [60]:
kernel.shape

(39, 39)

In [61]:
spatial_sizes

array([4.7385367 , 5.96982292, 2.93153839, ..., 4.27969986, 6.45006124,
       3.35932572])

In [ ]:
all_spatial_responses = []
neuron_ids = []
all_non_dom_spatial_responses = []

for i in range(len(spatial_range) - 1):
    sel = np.logical_and(
        spatial_sizes < spatial_range[i + 1], spatial_sizes >= spatial_range[i])
    if np.sum(sel) <= 0:
        continue
    neuron_ids.extend(np.where(sel)[0])